
**Audio Transcription and Summarization with FastAPI **

Name:Shivani Gunjal



Observation:Creating a system that can accept audio files, extract timestamps, summarize the information, and transcribing them is your duty. The system must be saved locally. You will also set up a FastAPI server to manage the process's endpoints.


In [17]:
!pip install fastapi uvicorn whisper transformers pydub nest_asyncio

In [18]:
!apt-get install ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [19]:
import nest_asyncio
from fastapi import FastAPI, UploadFile, File, HTTPException
from fastapi.responses import JSONResponse
import whisper
from transformers import pipeline
import os
import uvicorn
import warnings
import asyncio
from pydub import AudioSegment


nest_asyncio.apply()


app = FastAPI()


UPLOAD_DIR = "./uploads"
RESULTS_DIR = "./results"
os.makedirs(UPLOAD_DIR, exist_ok=True)
os.makedirs(RESULTS_DIR, exist_ok=True)

def preprocess_audio(file_location: str) -> str:
    try:

        audio = AudioSegment.from_file(file_location)

        processed_dir = os.path.join("uploads", "_processed")
        if not os.path.exists(processed_dir):
            os.makedirs(processed_dir)

        base_name = os.path.basename(file_location)
        processed_location = os.path.join(processed_dir, f"{os.path.splitext(base_name)[0]}_processed.wav")

        audio.export(processed_location, format="wav")
        return processed_location
    except Exception as e:
        raise RuntimeError(f"Failed to preprocess audio: {str(e)}")

warnings.filterwarnings("ignore", message="FP16 is not supported on CPU; using FP32 instead")
warnings.filterwarnings("ignore", message="Using a GPU for inference is not supported; using CPU instead")

try:

    whisper_model = whisper.load_model("large-v3")
    print("Whisper model loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Failed to load Whisper model: {str(e)}")

try:

    summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
    print("Summarization model loaded successfully.")
except Exception as e:
    raise RuntimeError(f"Failed to load summarization model: {str(e)}")

transcriptions = {}


100%|█████████████████████████████████████| 2.88G/2.88G [02:23<00:00, 21.5MiB/s]


Whisper model loaded successfully.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Summarization model loaded successfully.


In [ ]:
@app.post("/transcribe/")
async def transcribe_audio(file: UploadFile = File(...)):
    file_location = os.path.join(UPLOAD_DIR, file.filename)
    try:

        with open(file_location, "wb") as buffer:
            content = await file.read()
            buffer.write(content)


        processed_location = preprocess_audio(file_location)


        loop = asyncio.get_event_loop()
        transcription_result = await loop.run_in_executor(None, whisper_model.transcribe, processed_location)
        transcription = transcription_result['text']


        transcription_path = os.path.join(RESULTS_DIR, f"{file.filename}_transcription.txt")
        with open(transcription_path, "w") as f:
            f.write(transcription)


        transcriptions[file.filename] = transcription
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Transcription error: {str(e)}")
    finally:

        os.remove(file_location)
        if os.path.exists(processed_location):
            os.remove(processed_location)

    return {"session_id": file.filename, "transcription": transcription}

@app.post("/summarize/")
async def summarize_text(session_id: str):

    transcription = transcriptions.get(session_id)
    if not transcription:
        raise HTTPException(status_code=404, detail="Transcription not found for the given session ID")

    try:

        summary = summarizer(transcription, max_length=100, min_length=25, do_sample=False)[0]['summary_text']


        summary_path = os.path.join(RESULTS_DIR, f"{session_id}_summary.txt")
        with open(summary_path, "w") as f:
            f.write(summary)
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Summarization error: {str(e)}")

    return {"summary": summary}

@app.post("/extract-timestamps/")
async def extract_timestamps(session_id: str):

    transcription = transcriptions.get(session_id)
    if not transcription:
        raise HTTPException(status_code=404, detail="Transcription not found for the given session ID")

    try:

        file_location = os.path.join(UPLOAD_DIR, session_id)
        processed_location = preprocess_audio(file_location)
        loop = asyncio.get_event_loop()
        transcription_result = await loop.run_in_executor(None, whisper_model.transcribe, processed_location)
        segments = transcription_result['segments']


        timestamps = [{"start": seg['start'], "end": seg['end'], "text": seg['text']} for seg in segments]


        timestamps_path = os.path.join(RESULTS_DIR, f"{session_id}_timestamps.json")
        with open(timestamps_path, "w") as f:
            f.write(str(timestamps))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Timestamp extraction error: {str(e)}")

    return {"timestamps": timestamps}

@app.post("/process-audio/")
async def process_audio(session_id: str):

    transcription = transcriptions.get(session_id)
    if not transcription:
        raise HTTPException(status_code=404, detail="Transcription not found for the given session ID")

    try:

        summary = summarizer(transcription, max_length=100, min_length=25, do_sample=False)[0]['summary_text']


        file_location = os.path.join(UPLOAD_DIR, session_id)
        processed_location = preprocess_audio(file_location)

        loop = asyncio.get_event_loop()
        transcription_result = await loop.run_in_executor(None, whisper_model.transcribe, processed_location)
        segments = transcription_result['segments']
        timestamps = [{"start": seg['start'], "end": seg['end'], "text": seg['text']} for seg in segments]

        summary_path = os.path.join(RESULTS_DIR, f"{session_id}_summary.txt")
        timestamps_path = os.path.join(RESULTS_DIR, f"{session_id}_timestamps.json")

        with open(summary_path, "w") as f:
            f.write(summary)
        with open(timestamps_path, "w") as f:
            f.write(str(timestamps))
    except Exception as e:
        raise HTTPException(status_code=500, detail=f"Processing error: {str(e)}")

    return {"transcription": transcription, "summary": summary, "timestamps": timestamps}

if __name__ == "__main__":
    uvicorn.run(app, host="0.0.0.0", port=8000)


INFO:     Started server process [5196]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


Information:
1. FastAPI: FastAPI is a modern, fast (high-performance) web framework for building APIs with Python 3.6+ based on standard Python type hints.

2. Uvicorn: Uvicorn is a lightning-fast ASGI server implementation, using uvloop and httptools.

3. Whisper: Whisper is an automatic speech recognition (ASR) system developed by OpenAI.
Use Cases:
Transcribing audio into text.
Building voice-enabled applications.
Processing and analyzing audio data.

4. Transformers: Transformers is a library developed by Hugging Face that provides thousands of pre-trained models to perform tasks on texts such as classification, information extraction, question answering, summarization, translation, and more.

5. Pydub: Pydub is a simple and easy-to-use library that allows you to manipulate audio files with Python.

6. Nest-asyncio: Nest-asyncio is a small utility that allows you to use asyncio.run in the context of an already running event loop.

7. ffmpeg(FastAPI + Uvicorn):These libraries are not directly related to ffmpeg, but you might use ffmpeg in a FastAPI application to process multimedia files
